In [4]:
### This script will plot some graphics showing the relative locations of the ZDR and KDP maxes in the storm.
import matplotlib.pyplot as plt
import pyart
import numpy as np
import numpy.ma as ma
from metpy.units import atleast_1d, check_units, concatenate, units
from matplotlib.patches import PathPatch
from matplotlib.path import Path
from siphon.radarserver import RadarServer
#rs = RadarServer('http://thredds-aws.unidata.ucar.edu/thredds/radarServer/nexrad/level2/S3/')
rs = RadarServer('http://thredds.ucar.edu/thredds/radarServer/nexrad/level2/IDD/')
from datetime import datetime, timedelta
from siphon.cdmr import Dataset
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
from metpy.units import atleast_1d, check_units, concatenate, units
from shapely.geometry import polygon as sp
import pyproj 
import shapely.ops as ops
from shapely.ops import transform
from shapely.geometry.polygon import Polygon
from functools import partial
from shapely import geometry
import netCDF4

In [19]:
query = rs.query()
#Here, set the initial time of the archived radar loop you want.
dt = datetime(2017, 8, 26, 4, 10) # Our specified time
query.stations('KHGX').time_range(dt, dt + timedelta(hours=.8))
cat = rs.get_catalog(query)
cat.datasets
f = 27
n = 1
for item in sorted(cat.datasets.items()):
    # After looping over the list of sorted datasets, pull the actual Dataset object out
    # of our list of items and access over CDMRemote
    try:
        ds = item[1]
        radar = pyart.io.nexrad_cdm.read_nexrad_cdm(ds.access_urls['OPENDAP'])
        time_start = netCDF4.num2date(radar.time['data'][0], radar.time['units'])
        #Now let's calculate and plot specific differential phase.
        radar = radar.extract_sweeps([0])
        kdp_dict = pyart.retrieve.kdp_proc.kdp_maesaka(radar)
        radar.add_field('KDP', kdp_dict[0])

        # mask out last 10 gates of each ray, this removes the "ring" around the radar.
        radar.fields['differential_reflectivity']['data'][:, -10:] = np.ma.masked

        # exclude masked gates from the gridding
        gatefilter = pyart.filters.GateFilter(radar)
        gatefilter.exclude_masked('differential_reflectivity')

        #Now let's grid the data
        grid = pyart.map.grid_from_radars(
            (radar,), gatefilters=(gatefilter, ),
            grid_shape=(1, 500, 500),
            grid_limits=((200, 200), (-123000.0, 123000.0), (-123000.0, 123000.0)),
            fields=['differential_reflectivity','reflectivity','KDP'])

        #Get the data from the grid
        ZDR = grid.fields['differential_reflectivity']['data'][0]
        REF = grid.fields['reflectivity']['data'][0]
        KDP = grid.fields['KDP']['data'][0]

    #Mask everything below 20dbz
    #import numpy.ma as ma
        ZDRmasked = ma.masked_where(REF < 20, ZDR)
        REFmasked = ma.masked_where(REF < 20, REF)
        KDPmasked = ma.masked_where(REF < 20, KDP)

        rlons = grid.point_longitude['data']
        rlats = grid.point_latitude['data']
        cenlat = radar.latitude['data'][0]
        cenlon = radar.longitude['data'][0]
        #Let's set up the map projection!
        # Set up our projection
        crs = ccrs.LambertConformal(central_longitude=-100.0, central_latitude=45.0)

        # Set up our array of latitude and longitude values and transform to 
        # the desired projection.

        tlatlons = crs.transform_points(ccrs.LambertConformal(central_longitude=265, central_latitude=25, standard_parallels=(25.,25.)),rlons[0,:,:],rlats[0,:,:])
        tlons = tlatlons[:,:,0]
        tlats = tlatlons[:,:,1]

        # Limit the extent of the map area, must convert to proper coords.
        LL = (cenlon-1,cenlat-1,ccrs.PlateCarree())
        UR = (cenlon+1,cenlat+1,ccrs.PlateCarree())
        print(LL)

        # Get data to plot state and province boundaries
        states_provinces = cfeature.NaturalEarthFeature(
                category='cultural',
                name='admin_1_states_provinces_lakes',
                scale='50m',
                facecolor='none')
        fname = 'cb_2016_us_county_20m/cb_2016_us_county_20m.shp'
        fname2 = 'cb_2016_us_state_20m/cb_2016_us_state_20m.shp'
        counties = ShapelyFeature(Reader(fname).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
        states = ShapelyFeature(Reader(fname2).geometries(),ccrs.PlateCarree(), facecolor = 'none', edgecolor = 'black')
        fig=plt.figure(n,figsize=(30.,25.))
        ax = plt.subplot(111,projection=ccrs.PlateCarree())
        ax.coastlines('50m',edgecolor='black',linewidth=0.75)
        #ax.add_feature(states_provinces,edgecolor='black',linewidth=0.5)
        ax.add_feature(counties, edgecolor = 'black', linewidth = 0.5)
        ax.add_feature(states, edgecolor = 'black', linewidth = 1.5)
        ax.set_extent([LL[0],UR[0],LL[1],UR[1]])
        REFlevels = np.arange(20,73,2)
        ax.contourf(rlons[0,:,:],rlats[0,:,:],REFmasked,REFlevels,cmap = plt.cm.gist_ncar)
        ZDRlevels = np.arange(3, np.max(ZDRmasked)+((np.max(ZDRmasked))-3)/2, (np.max(ZDRmasked))-3)
        zdrlev = [1.5]
        kdplev = [.75]
        #ZDRlevels = np.arange(3,5.5,.5)
        #ZDRlevels1 = np.arange(5,10,.5)
        KDPlevels = np.arange(.75, np.max(KDPmasked)+((np.max(KDPmasked))-1.5)/2, (np.max(KDPmasked))-1.5)
        #KDPlevels = np.arange(.75,1.75,.25)
        #KDPlevels1 = np.arange(1.5,10,.25)
        zdrc = ax.contour(rlons[0,:,:],rlats[0,:,:],ZDRmasked,zdrlev,linewidths = 5, colors='blue', alpha = .8)
        #zrdc = ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,ZDRlevels,linewide = .01, colors='pink', alpha = .8)
        #ax.contourf(rlons[0,:,:],rlats[0,:,:],ZDRmasked,ZDRlevels1,linewide = .01, colors='crimson', alpha = .8)

        #kdpc = ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels,linewide = .01, colors ='green', alpha = .5)
        kdpc = ax.contour(rlons[0,:,:],rlats[0,:,:],KDPmasked,kdplev,linewidths = 5, colors='green', alpha = .8)
        #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
        plt.savefig('testfig.png')

        proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
                   pyproj.Proj(init='epsg:3857'))
        zdr_areas = []
        zdr_centroid_lon = []
        zdr_centroid_lat = []
        zdr_mean = []
        zdr_max = []

        for col in zdrc.collections:
            # Loop through all polygons that have the same intensity level
            print('hi')
            for contour_path in col.get_paths(): 
                # Create the polygon for this intensity level
                # The first polygon in the path is the main one, the following ones are "holes"
                print('hi')
                for ncp,cp in enumerate(contour_path.to_polygons()):
                    print('hi')
                    cpa = np.asarray(cp[:])
                    x = cpa[:,0]
                    y = cpa[:,1]
                    new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                    if ncp == 0:
                        poly = new_shape
                        #print('hi')
                    else:
                        # Remove the holes if there are any
                        poly = poly.difference(new_shape)
                        #print('hi')

                # do something with polygon
                #print(poly.area) 
                #print(poly.centroid.x)
                s_new = transform(proj, poly)
                projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                boundary = np.asarray(poly.boundary.xy)
                polypath = Path(boundary.transpose())
                coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
                mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
                mean = np.mean(ZDRmasked[mask])
                if projected_area > 20 * units('km^2') and mean > 1.5:
                    zdr_areas.append((projected_area))
                    zdr_centroid_lon.append((poly.centroid.x))
                    zdr_centroid_lat.append((poly.centroid.y))
                    zdr_mean.append((mean))
                    zdr_max.append((np.max(ZDRmasked[mask])))
                #print(s_new)

        kdp_areas = []
        kdp_centroid_lon = []
        kdp_centroid_lat = []

        for col in kdpc.collections:
            # Loop through all polygons that have the same intensity level
            for contour_path in col.get_paths(): 
                # Create the polygon for this intensity level
                # The first polygon in the path is the main one, the following ones are "holes"
                for ncp,cp in enumerate(contour_path.to_polygons()):
                    print(1)
                    cpa = np.asarray(cp[:])
                    x = cpa[:,0]
                    y = cpa[:,1]
                    new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
                    if ncp == 0:
                        poly = new_shape
                    else:
                        # Remove the holes if there are any
                        poly = poly.difference(new_shape)

                # do something with polygon
                #print(poly.area) 
                print(poly.centroid.x)
                s_new = transform(proj, poly)
                projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
                if projected_area > 20 * units('km^2'):
                    kdp_areas.append((projected_area))
                    kdp_centroid_lon.append((poly.centroid.x))
                    kdp_centroid_lat.append((poly.centroid.y))

            #print(s_new)

        if np.asarray(zdr_centroid_lon).shape[0] > 0:
            ax.scatter(zdr_centroid_lon, zdr_centroid_lat, marker = '*', s = 500, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
        if np.asarray(kdp_centroid_lon).shape[0] > 0:
            ax.scatter(kdp_centroid_lon, kdp_centroid_lat, marker = '^', s = 500, color = 'black', zorder = 10, transform=ccrs.PlateCarree())
        for i in enumerate(zdr_areas):
            plt.text(zdr_centroid_lon[i[0]]+.016, zdr_centroid_lat[i[0]]+.016, "%.2f / %.2f / %.2f" %(zdr_areas[i[0]].magnitude, zdr_mean[i[0]], zdr_max[i[0]]), size = 23)
            #plt.annotate(zdr_areas[i[0]], (zdr_centroid_lon[i[0]],zdr_centroid_lat[i[0]]))
        #ax.contourf(rlons[0,:,:],rlats[0,:,:],KDPmasked,KDPlevels1,linewide = .01, colors ='b', alpha = .5)
        #plt.tight_layout()
        #plt.savefig('ZDRarcannotated.png')
        title_plot = plt.title('Radar Reflectivity, ZDR, and KDP '+str(time_start.year)+'-'+str(time_start.month)+'-'+str(time_start.day)+
                                   ' '+str(time_start.hour)+':'+str(time_start.minute)+' UTC', size = 25)
        plt.savefig('ZDRArcFinderTropicalKHGX'+str(time_start.year)+str(time_start.month)+str(time_start.day)+str(time_start.hour)+'0'+str(time_start.minute)+'.png')
        n = n+1
    except:
        continue

plt.show()

C:\Users\matts\Miniconda3\envs\devel\lib\site-packages\pyart\io\nexrad_cdm.py:141: RuntimeWarning: invalid value encountered in greater
  azim_data[ray_i:end] = dvars[azimuth_var][var_index][:nradials]
C:\Users\matts\Miniconda3\envs\devel\lib\site-packages\pyart\io\nexrad_cdm.py:142: RuntimeWarning: invalid value encountered in greater
  elev_data[ray_i:end] = dvars[elevation_var][var_index][:nradials]
C:\Users\matts\Miniconda3\envs\devel\lib\site-packages\pyart\io\nexrad_cdm.py:144: RuntimeWarning: invalid value encountered in greater
  dvars[elevation_var][var_index][:nradials])


In [6]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
zdr_areas = []
zdr_centroid_lon = []
zdr_centroid_lat = []
zdr_mean = []
zdr_max = []

for col in zdrc.collections:
    # Loop through all polygons that have the same intensity level
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        for ncp,cp in enumerate(contour_path.to_polygons()):
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
                print(new_shape)
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)

        # do something with polygon
        #print(poly.area) 
        print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        boundary = np.asarray(poly.boundary.xy)
        polypath = Path(boundary.transpose())
        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
        mean = np.mean(ZDRmasked[mask])
        if projected_area > 20 * units('km^2') and mean > 3.0:
            zdr_areas.append((projected_area))
            zdr_centroid_lon.append((poly.centroid.x))
            zdr_centroid_lat.append((poly.centroid.y))
            zdr_mean.append((mean))
            zdr_max.append((np.max(ZDRmasked[mask])))
        #print(s_new)
print(zdr_areas)
print(zdr_centroid_lon)
print(zdr_centroid_lat)
print(zdr_mean)
print(zdr_max)

POLYGON ((-87.14650269865113 32.82473736538209, -87.14654868829419 32.82917140303287, -87.14700313010604 32.8336093530376, -87.14618932442924 32.83803515542336, -87.14426907178432 32.84095094193749, -87.14348134826658 32.84244278025931, -87.14320003794761 32.84687367235605, -87.14400880658687 32.85131505357682, -87.14412653817635 32.85152746337874, -87.14523285494377 32.85576038189856, -87.14504025932081 32.86019213395635, -87.14588613693429 32.86463383537282, -87.14721510949792 32.86908016611732, -87.1484274403183 32.87352538031153, -87.14909014961773 32.87491189447746, -87.15042181754164 32.87797803027838, -87.15396458257069 32.88244542354081, -87.15426466374069 32.8827220207276, -87.15949797148092 32.88618561136618, -87.16476890509351 32.88684218515808, -87.1658554374341 32.88699164220916, -87.17003894155998 32.88757239816948, -87.17529764957479 32.88917123609809, -87.18057662112363 32.88923522403788, -87.18585960480704 32.88899139722734, -87.19115226210907 32.88799858675585, -87.19

POLYGON ((-88.09297598034306 33.25113103845684, -88.09306849802455 33.251362761465, -88.09466160264948 33.25579930366005, -88.09782969654424 33.26023882019854, -88.09825200268629 33.26069934043359, -88.1035439340432 33.26458269344208, -88.10364788675865 33.26468310515776, -88.10401862252995 33.26911730722803, -88.10352888139766 33.2703836938766, -88.10266735443197 33.27354836845205, -88.10036086525729 33.2779776455526, -88.09819794627087 33.2810501449646, -88.09695689196344 33.28240484350101, -88.09673742279992 33.28683796327235, -88.09724214437723 33.29127245089927, -88.0981683266091 33.29219408936279, -88.10346607046412 33.29457541552168, -88.10822242613163 33.29572608974299, -88.10876698771987 33.2958201217169, -88.10943521276408 33.29572824779169, -88.11407301738535 33.29506073053361, -88.11937936887304 33.2941293338919, -88.12468642105995 33.29285153882661, -88.12999365011534 33.29142819199448, -88.13076803897023 33.29133076211241, -88.13529954124846 33.29052621537562, -88.1406079

POLYGON ((-88.54143296330368 33.82333083902631, -88.54503057723434 33.82490284498289, -88.54616983683933 33.82332261821379, -88.54604412807129 33.82247568232404, -88.54143296330368 33.82333083902631))
-88.54432913140134
POLYGON ((-88.24642928224151 33.88112183072212, -88.2470939186511 33.87668872913978, -88.24611846153718 33.87607396403728, -88.24435668223161 33.87668688159805, -88.24077543415179 33.87897755308873, -88.23824169671789 33.88111608258338, -88.23606131227778 33.8855480150527, -88.23933144607707 33.88674242436772, -88.24642928224151 33.88112183072212))
-88.24189931341498
POLYGON ((-88.22590956316562 33.88997350915045, -88.22474305728649 33.88859755475809, -88.22445029806056 33.88997230015734, -88.22464390161267 33.89005329766938, -88.22590956316562 33.88997350915045))
-88.22503247090957
POLYGON ((-88.76957192675654 33.90113500799949, -88.76721520521798 33.90254080427768, -88.76765417881073 33.90414377117036, -88.76957192675654 33.90113500799949))
-88.76814710359508
POLYGON 

[]
[]
[]
[]
[]


In [7]:
for i in range(len(zdrc.collections)):
    p = zdrc.collections[i].get_paths()[0]
    v = p.vertices
    x = v[:,0]
    y = v[:,1]
    if len(x)>2:
        poly = sp.Polygon([(i[0], i[1]) for i in zip(x,y)])
        print(i, poly.area)

0 0.003594768552302712


In [8]:
print(len(zdrc.collections))

1


In [9]:
print(enumerate(contour_path.to_polygons()))
for cp in enumerate(contour_path.to_polygons()):
    print(1)

1


In [10]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
kdp_areas = []
kdp_centroid_lon = []
kdp_centroid_lat = []

from shapely import geometry
for col in kdpc.collections:
    # Loop through all polygons that have the same intensity level
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        for ncp,cp in enumerate(contour_path.to_polygons()):
            print(1)
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)

        # do something with polygon
        #print(poly.area) 
        print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        if projected_area > 20 * units('km^2'):
            kdp_areas.append((projected_area))
            kdp_centroid_lon.append((poly.centroid.x))
            kdp_centroid_lat.append((poly.centroid.y))

        #print(s_new)
print(kdp_areas)
print(kdp_centroid_lon)
print(kdp_centroid_lat)

1
-88.2279510443226
1
-88.92762973301487
1
-88.90014267895808
1
-88.86370845890099
1
-89.2454798303056
1
-89.27887977518918
[<Quantity(282.68846704195624, 'kilometer ** 2')>, <Quantity(50.18036435956034, 'kilometer ** 2')>, <Quantity(192.25203767886327, 'kilometer ** 2')>, <Quantity(39.52924508427346, 'kilometer ** 2')>]
[-88.92762973301487, -88.86370845890099, -89.2454798303056, -89.27887977518918]
[34.25133703035296, 34.45675677315477, 34.49475900749556, 34.62730337540038]


In [11]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))
zdr_areas = []
zdr_centroid_lon = []
zdr_centroid_lat = []
zdr_mean = []
zdr_max = []

for col in zdrc.collections:
    # Loop through all polygons that have the same intensity level
    print('hi')
    for contour_path in col.get_paths(): 
        # Create the polygon for this intensity level
        # The first polygon in the path is the main one, the following ones are "holes"
        print('hi')
        for ncp,cp in enumerate(contour_path.to_polygons()):
            print('hi')
            cpa = np.asarray(cp[:])
            x = cpa[:,0]
            y = cpa[:,1]
            new_shape = geometry.Polygon([(i[0], i[1]) for i in zip(x,y)])
            if ncp == 0:
                poly = new_shape
                print('hi')
            else:
                # Remove the holes if there are any
                poly = poly.difference(new_shape)
                print('hi')

        # do something with polygon
        #print(poly.area) 
        #print(poly.centroid.x)
        s_new = transform(proj, poly)
        projected_area = (transform(proj, poly).area * units('m^2')).to('km^2')
        boundary = np.asarray(poly.boundary.xy)
        polypath = Path(boundary.transpose())
        coord_map = np.vstack((rlons[0,:,:].flatten(), rlats[0,:,:].flatten())).T # create an Mx2 array listing all the coordinates in field
        mask = polypath.contains_points(coord_map).reshape(rlons[0,:,:].shape)
        mean = np.mean(ZDRmasked[mask])
        if projected_area > 20 * units('km^2') and mean > 3.0:
            zdr_areas.append((projected_area))
            zdr_centroid_lon.append((poly.centroid.x))
            zdr_centroid_lat.append((poly.centroid.y))
            zdr_mean.append((mean))
            zdr_max.append((np.max(ZDRmasked[mask])))
        #print(s_new)
print(zdr_areas)
print(zdr_centroid_lon)
print(zdr_centroid_lat)
print(zdr_mean)
print(zdr_max)

hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
[]
[]
[]
[]
[]
